# Наука о подацима у облаку: "Azure ML SDK" приступ

## Увод

У овом нотебуку ћемо научити како да користимо Azure ML SDK за тренирање, постављање и коришћење модела преко Azure ML.

Предуслови:
1. Направили сте Azure ML радни простор.
2. Учитали сте [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) у Azure ML.
3. Отпремили сте овај нотебук у Azure ML Studio.

Следећи кораци су:

1. Направите експеримент у постојећем радном простору.
2. Направите кластер за рачунање.
3. Учитајте скуп података.
4. Конфигуришите AutoML користећи AutoMLConfig.
5. Покрените AutoML експеримент.
6. Истражите резултате и пронађите најбољи модел.
7. Региструјте најбољи модел.
8. Поставите најбољи модел.
9. Користите крајњу тачку.

## Увоз специфичан за Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Иницијализација радног окружења  
Иницијализујте објекат радног окружења из сачуване конфигурације. Уверите се да је конфигурациона датотека присутна на .\config.json  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Креирајте Azure ML експеримент

Хајде да креирамо експеримент под именом 'aml-experiment' у радном простору који смо управо иницијализовали.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Креирајте кластер за рачунање  
Биће потребно да креирате [рачунарски циљ](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) за ваше AutoML покретање.  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Подаци  
Уверите се да сте отпремили скуп података у Azure ML и да је кључ истог имена као и скуп података.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## АутоМЛ Конфигурација


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## АутоМЛ Рун


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Деплојовање најбољег модела

Покрените следећи код да бисте деплојовали најбољи модел. Стање деплојовања можете видети у Azure ML порталу. Овај корак може трајати неколико минута.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Коришћење крајње тачке
Можете додати уносе у следећи пример уноса.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако тежимо тачности, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.  
